# MOVIE RECOMMENDATION SYSTEM

## Overview

## <small> Business Understanding</small>

### <small> Introduction</small>

### <small>  Problem Statement</small>

With so many movie alternatives available nowadays, people find it difficult to choose films that suit their tastes due to information overload. Manual searches take a lot of time, which causes decision fatigue and lowers user engagement on a platform. A customized movie recommendation system that makes use of user-specific information and collaborative filtering methods will address this. By creating an effective recommendation engine that smoothly connects with user profiles, this project seeks to improve user satisfaction and retention by enabling a more efficient and enjoyable exploration of the extensive movie catalogs.

### <small> Main Objective</small>

### <small> Subjective Objectives</small>

### <small>  Data Understanding</small>

##  Reading the data

In [1]:
# Importing the necessary libraries 
import pandas as pd # For data manipulation and analysis
import numpy as np # For numerical operations and array manipulation
import matplotlib.pyplot as plt #Data Visualisation
%matplotlib inline
import warnings
import seaborn as sns # for enhanced visualisation

In [3]:
# Loading the dataset
df = pd.read_csv("tags.csv")
# Preview the shape of the dataset
print('Dataframe shape {}'.format(df.shape))

FileNotFoundError: [Errno 2] No such file or directory: 'tags.csv'